In [4]:
%cd /Users/deeprodge/Downloads/DREAMS/PG&E/rock_detection_3d/unsupervised_rock_detection_3d

/Users/deeprodge/Downloads/DREAMS/PG&E/rock_detection_3d/unsupervised_rock_detection_3d


In [26]:
import laspy
import os
import open3d as o3d
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [27]:
las_files = ['box_pbr/' + f for f in os.listdir('box_pbr/') if f.endswith('.las')]
las_file_path = las_files[0]
print(las_file_path)
# Read .las file using laspy
# las = laspy.read(las_files[1])
# point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))
# point_data
# geom = o3d.geometry.PointCloud()
# geom.points = o3d.utility.Vector3dVector(point_data)
# o3d.visualization.draw_geometries([geom])



# for las_file in las_files[:5]:
pc = laspy.read(las_files[0])
x = pc.x.scaled_array()
x_mean = np.mean(x)
y = pc.y.scaled_array()
y_mean = np.mean(y)
z = pc.z.scaled_array()
z_mean = np.mean(z)
r = np.uint8(pc.red/65535.*255)
g = np.uint8(pc.green/65535.*255)
b = np.uint8(pc.blue/65535.*255)
rgb = np.vstack((r, g, b)).transpose()
xyz = np.vstack((x-x_mean, y-y_mean, z-z_mean)).transpose()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(rgb)
o3d.io.write_point_cloud("pbr_mean.pcd", pcd)
#o3d.visualization.draw_geometries([pcd])
#     print(las_file)

box_pbr/pbr107.las


True

In [335]:
class KMeansSegmentation:
    def __init__(self, pcd, n_clusters=2, height_weight=1.0, slope_weight=1.0, normal_weight=1.0):
        self.pcd = pcd
        self.n_clusters = n_clusters
        self.labels = np.array([])
        self.height_weight = height_weight
        self.slope_weight = slope_weight
        self.normal_weight = normal_weight

        voxel_size = 0.01
        self.pcd = self.pcd.voxel_down_sample(voxel_size)
        radius_normal = voxel_size * 10
        self.pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=100))
        self.pcd_tree = o3d.geometry.KDTreeFlann(self.pcd)


        
        # Adjusting the orientations of the normals to be consistent with each other, based on local tangent planes.
        self.pcd.orient_normals_consistent_tangent_plane(k=4)
        self.pcd.normals = o3d.utility.Vector3dVector(-np.asarray(self.pcd.normals))

    def calculate_slope(self, normals):
        # Assuming that the normals are already normalized
        # The slope can be calculated as the arccosine of the dot product
        # of the normal with the Z-axis (0, 0, 1)
        z_axis = np.array([0, 0, 1])
        slopes = np.arccos(np.clip(np.dot(normals, z_axis), -1.0, 1.0))
        return slopes

    def calculate_slope(self, normals):
        points = np.asarray(self.pcd.points)
        normals = self.normalize(np.asarray(normals),type_of_normalization = "minmax")
        
        slopes = np.zeros(len(points))

        for i, point in enumerate(points):
            k, idx, _ = self.pcd_tree.search_radius_vector_3d(point, 0.2)

            if k > 1:  # More than one point in the neighborhood
                neighbors_normals = normals[idx, :]
                current_normal = normals[i]

                # Calculate dot product
                dot_products = np.clip(np.dot(neighbors_normals, current_normal), -1.0, 1.0)
                min_dot = np.min(dot_products)
                slopes[i] = min_dot
        
        return slopes

    def calculate_curvature(self):

        # Placeholder for curvature values
        curvature = np.zeros(len(np.asarray(self.pcd.points)))

        for i, point in enumerate(self.pcd.points):
            # Search for nearest neighbors of the given point
            [k, idx, _] = self.pcd_tree.search_radius_vector_3d(point, 0.5)  # Radius 
            
            # Calculate the covariance matrix of the neighborhood
            if k < 3:
                continue
            
            neighbors = np.asarray(self.pcd.points)[idx, :]
            mean = np.mean(neighbors, axis=0)
            covariance_matrix = np.cov((neighbors - mean).T)

            # Eigen decomposition
            eigen_values, _ = np.linalg.eigh(covariance_matrix)
            eigen_values.sort()

            # The curvature can be approximated as the ratio of the smallest
            # to the sum of eigenvalues (Gaussian curvature approximation)
            curvature[i] = eigen_values[0] / np.sum(eigen_values)
        
        return curvature

    def normalize(self, data, type_of_normalization = "standarization"):
        min_val = np.min(data)
        max_val = np.max(data)
        if type_of_normalization == "minmax":
            return (data - min_val) / (max_val - min_val)
        elif type_of_normalization == "standarization":
            return (data - np.mean(data))/np.std(data)
    
    def segment(self):

        type_of_normalization = "minmax"
        
        pcd_points = np.asarray(self.pcd.points)
        normals = np.asarray(self.pcd.normals)

        # Normalize the features
        normals = self.normalize(normals, type_of_normalization = type_of_normalization)
        min_z = np.min(pcd_points[:, 2])
        relative_height = pcd_points[:, 2] - min_z
        relative_height = self.normalize(relative_height, type_of_normalization = type_of_normalization)
        slopes = self.calculate_slope(normals)
        slopes = self.normalize(slopes, type_of_normalization = type_of_normalization)
        x_norm = pcd_points[:, 0] #self.normalize(pcd_points[:, 0], type_of_normalization = type_of_normalization)
        y_norm = pcd_points[:, 1] #self.normalize(pcd_points[:, 1], type_of_normalization = type_of_normalization)

        # x_norm = self.normalize(pcd_points[:, 0], type_of_normalization = type_of_normalization)
        # y_norm = self.normalize(pcd_points[:, 1], type_of_normalization = type_of_normalization)
        
        # Apply feature weights
        relative_height *= self.height_weight
        slopes *= self.slope_weight
        normals *= self.normal_weight

        # Combine features with weights applied
        features = np.hstack((x_norm[:, np.newaxis],
                              y_norm[:, np.newaxis],
                              relative_height[:, np.newaxis],
                              slopes[:, np.newaxis],
                              normals))

        # Apply K-Means
        kmeans = KMeans(n_clusters=self.n_clusters)
        kmeans.fit(features)

        # Assign clusters
        self.labels = kmeans.labels_
        
        return self.labels

        # Normalize the features all at once
        # features = (features - np.mean(features, axis=0)) / np.std(features, axis=0)

        # Apply K-Means
        kmeans = KMeans(n_clusters=self.n_clusters)
        kmeans.fit(features)

        # Assign clusters
        self.labels = kmeans.labels_
        
        return self.labels


    def visualize_segmentation(self):
        max_label = self.labels.max()
        colors = plt.get_cmap('viridis')(self.labels / (max_label if max_label > 0 else 1))
        colors = colors[:, :3]  # remove the alpha channel
        self.pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([self.pcd],
                                          #point_show_normal = True
                                         )

In [345]:

# Load your point cloud
pcd = o3d.io.read_point_cloud("box_pbr/pbr91.pcd")


# Create an instance of the KMeansSegmentation class
kmeans_segmentation = KMeansSegmentation(pcd, n_clusters=2,
                                         #height_weight=2.5, slope_weight=2.0, normal_weight=3, #best for global slope and standardization
                                         height_weight=6, slope_weight=6, normal_weight=2.5 #best for global slope and minmax
                                        )

# Perform K-Means segmentation
segment_labels = kmeans_segmentation.segment()

# kmeans_segmentation.post_process_segments(
#     #cluster_size_threshold_ratio=0.15
# )
# Visualize the segmented point cloud
kmeans_segmentation.visualize_segmentation()

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [249]:
np.linalg.norm(np.asarray(kmeans_segmentation.pcd.normals),axis = 1)

array([2.5, 2.5, 2.5, ..., 2.5, 2.5, 2.5])

In [243]:
import laspy
import numpy as np

# Read LAS file
pc = laspy.read("box_pbr/pbr42.las")
x = pc.x.scaled_array()
x_mean = np.mean(x)
y = pc.y.scaled_array()
y_mean = np.mean(y)
z = pc.z.scaled_array()
z_mean = np.mean(z)
r = np.uint8(pc.red/65535.*255)
g = np.uint8(pc.green/65535.*255)
b = np.uint8(pc.blue/65535.*255)
rgb = np.vstack((r, g, b)).transpose()
xyz = np.vstack((x-x_mean, y-y_mean, z-z_mean)).transpose()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(rgb)

# Example: Calculating global slopes (you need to adjust this part based on your actual function)
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=100))
pcd.orient_normals_consistent_tangent_plane(k=4)
pcd.normals = o3d.utility.Vector3dVector(-np.asarray(pcd.normals))
normals = np.asarray(pcd.normals)
z_axis = np.array([0, 0, 1])

pcd_points = np.asarray(pcd.points)
min_z = np.min(pcd_points[:, 2])
relative_height = pcd_points[:, 2] - min_z

slopes = relative_height #np.arccos(np.clip(np.dot(normals, z_axis), -1.0, 1.0))
slopes = (slopes-np.mean(slopes))/(np.std(slopes))
print(slopes)

# points = np.asarray(pcd.points)
# slopes = np.zeros(len(points))
# pcd_tree = o3d.geometry.KDTreeFlann(pcd)

# for i, point in enumerate(points):
#     k, idx, _ = pcd_tree.search_radius_vector_3d(point, 0.2)  # You can adjust the radius as necessary

#     if k > 1:  # More than one point in the neighborhood
#         neighbors_normals = normals[idx, :]
#         current_normal = normals[i]

#         # Calculate angular difference
#         dot_products = np.clip(np.dot(neighbors_normals, current_normal), -1.0, 1.0)
#         min_dot = np.min(dot_products)
#         slopes[i] = min_dot


# Normalize slopes to fit into the uint16 range
slopes_normalized = np.interp(slopes, (np.min(slopes), np.max(slopes)), (0, 65535)).astype(np.uint16)

# Assign slopes to the intensity channel
pc.intensity = slopes_normalized

# Write the modified LAS file
pc.write("modified_file_global_slope_stand.las")


[-0.81285665 -0.80025564 -0.80025564 ...  0.97648644  0.97648644
  1.00168846]
